# **Execute this once**

In [ ]:
#@title
# section 1
%pip install --quiet --upgrade diffusers transformers accelerate mediapy scipy ftfy spacy

# section 2
# The xformers package is mandatory to be able to create several 768x768 images.
github_url = "https://github.com/brian6091/xformers-wheels"
xformer_id = "0.0.15.dev0+4c06c79"
xformers_wheels = f"xformers-{xformer_id}.d20221205-cp38-cp38-linux_x86_64.whl"

# Install xformers using pre-compiled Python wheels
%pip install -q {github_url}/releases/download/{xformer_id}/{xformers_wheels}


# section 3
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler, DPMSolverMultistepScheduler

scheduler = None
# scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler")


# section 4
# model_id = "stabilityai/stable-diffusion-2-1-base"
# model_id = "stabilityai/stable-diffusion-2-1"
model_id = "dreamlike-art/dreamlike-photoreal-2.0" # previous


# sections 4
import mediapy as media
import torch
import os
from diffusers import StableDiffusionPipeline

device = "cuda"

if model_id.startswith("stabilityai/"):
  model_revision = "fp16"
else:
  model_revision = None

if scheduler is None:
  pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      torch_dtype=torch.float16,
      revision=model_revision,
      )  
else:
  pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      scheduler=scheduler,
      torch_dtype=torch.float16,
      revision=model_revision,
      )

pipe = pipe.to(device)
pipe.enable_xformers_memory_efficient_attention()

if model_id.endswith('-base'):
  image_length = 512
else:
  image_length = 768


In [ ]:
prompt = "A detailed painting of a cute cyberpunk girl with green eyes, d.va, cyberpunk 2077, blade runner 2049, dark city background, nude"
remove_safety = True
num_images = 4

if remove_safety:
  negative_prompt = None
else:
  negative_prompt = "nude, naked"
    # pass

new_height = 768
new_width = 512


# 1 – Mostly ignore your prompt.
# 3 – Be more creative.
# 7 – A good balance between following the prompt and freedom.
# 15 – Adhere more to prompt.
# 30 – Strictly follow the prompt.

# guidance_scale = 9 # default
guidance_scale = 9

images = pipe(
    prompt,
    height = new_height, #image_length,
    width = new_width, #image_length,
    num_inference_steps = 50,
    guidance_scale = guidance_scale,
    num_images_per_prompt = num_images,
    negative_prompt = negative_prompt,
    ).images

media.show_images(images)
images[0].save("output.jpg")